In [2]:
!pip install datasets --q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 11.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


# Main NLLB Dataset

### Use to download when needed only. Takes a lot of time.

First downloading the dataset for English-Nepali Pair
- There are 19643845 rows.
- Upon manual inspections, we have found many translations to be vary inaccurate or containing languages other than nepali and english.
- We will filter it to include only quality translations.

In [3]:
from datasets import load_dataset

ds = load_dataset("allenai/nllb", 'eng_Latn-npi_Deva')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/38.6k [00:00<?, ?B/s]

nllb.py:   0%|          | 0.00/9.49k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/5.05M [00:00<?, ?B/s]

nllb_lang_pairs.py:   0%|          | 0.00/81.9k [00:00<?, ?B/s]

The repository for allenai/nllb contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/allenai/nllb.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Repo card metadata block was not found. Setting CardData to empty.


Generating train split:   0%|          | 0/19643845 [00:00<?, ? examples/s]

Loading dataset shards:   0%|          | 0/24 [00:00<?, ?it/s]

Inspecting few rows:

In [4]:
ds['train'][3]

{'translation': {'eng_Latn': 'the book of the book in which there is no doubt from the Lord of this world.',
  'npi_Deva': 'साहिल हो गई बल्ले बल्ले रब ने किते कम सवले'},
 'laser_score': 1.2496018409729004,
 'source_sentence_lid': 1.0000100135803223,
 'target_sentence_lid': 0.7389299869537354,
 'source_sentence_source': 'crawl-data/CC-MAIN-2019-39/segments/1568514573561.45/wet/CC-MAIN-20190919163337-20190919185337-00056.warc.wet.gz',
 'source_sentence_url': 'https://fr.scribd.com/document/29535404/Is-the-Quran-Gods-Word-Book-By-Dr-Zakir-Naik',
 'target_sentence_source': 'crawl-data/CC-MAIN-2020-50/segments/1606141747887.95/wet/CC-MAIN-20201205135106-20201205165106-00603.warc.wet.gz',
 'target_sentence_url': 'https://www.bhajanganga.com/bhajan/lyrics/id/19163/title/tera-mukhda-tk-tk-saiyan-lakh-lakh-shukar-manawagi'}

In [5]:
ds['train'][0]

{'translation': {'eng_Latn': 'than for their Lord."',
  'npi_Deva': 'सिर्फ परवरदिगार से"'},
 'laser_score': 1.2498674392700195,
 'source_sentence_lid': 0.9999600052833557,
 'target_sentence_lid': 0.7853299975395203,
 'source_sentence_source': 'crawl-data/CC-MAIN-2021-43/segments/1634323585381.88/wet/CC-MAIN-20211021040342-20211021070342-00088.warc.wet.gz',
 'source_sentence_url': 'https://thecatacombs.freeforums.net/thread/1910/devotion-sorrows-blessed-virgin-mary',
 'target_sentence_source': 'crawl-data/CC-MAIN-2021-43/segments/1634323587794.19/wet/CC-MAIN-20211026011138-20211026041138-00579.warc.wet.gz',
 'target_sentence_url': 'https://www.makelifefun.in/2021/10/here-are-some-most-famous-bollywood.html'}

Here we can see that the source and target sentence lid corresponds to how close the sentence is related to the source and target language. Here, we can see the score of 0.78 and we can clearly see that the sentence is not in Nepali. So, we will create a filter to ensure only Nepali and english sentences. For that, we will create a filter that allows lid > 0.95 only.

- Removing useless columns.

In [6]:
ds_columns_removed = ds['train'].remove_columns(['source_sentence_url', 'target_sentence_source', 'source_sentence_source', 'target_sentence_url'])

In [7]:
ds_columns_removed[0]

{'translation': {'eng_Latn': 'than for their Lord."',
  'npi_Deva': 'सिर्फ परवरदिगार से"'},
 'laser_score': 1.2498674392700195,
 'source_sentence_lid': 0.9999600052833557,
 'target_sentence_lid': 0.7853299975395203}

### Dataset Subset 1. First 500,000 data rows.

#### Using the cells below during saving only. Click on down arrow. Directly go to load

In [8]:
dataset_subset_1 = ds_columns_removed[0:500000]

In [9]:
import time

In [127]:
import pandas as pd
start_time = time.time()

# Load the first 1000 rows into a pandas DataFrame
subset_500000 = ds_columns_removed[19500000:]
df_500000= pd.DataFrame(subset_500000)

# End the timer
end_time = time.time()

# Calculate the elapsed time
elapsed_time = end_time - start_time

In [128]:
print(f'elapsed_time: {elapsed_time}')

elapsed_time: 1.8903288841247559


In [129]:
df_500000.to_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/nllb-subsets/nllb-subset-40.csv')

- It takes 5 seconds to load the 500000 data into pandas dataframe.

#### LOAD

In [1]:
import pandas as pd

In [2]:
df_500000 = pd.read_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/nllb-subsets/nllb-subset-1.csv')

In [3]:
df_500000.head(3)

,Unnamed: 0,translation,laser_score,source_sentence_lid,target_sentence_lid
0,0,"{'eng_Latn': 'than for their Lord.""', 'npi_Dev...",1.249867,0.99996,0.78533
1,1,"{'eng_Latn': 'Indeed, for he who fears the pun...",1.249681,0.99999,0.99941
2,2,"{'eng_Latn': 'On that day man shall cry, ""Wher...",1.249663,0.99998,0.99986


In [4]:
df_500000.columns

Index(['Unnamed: 0', 'translation', 'laser_score', 'source_sentence_lid',
       'target_sentence_lid'],
      dtype='object')

In [5]:
df_500000.drop(columns=['Unnamed: 0'], inplace=True)

In [6]:
df_500000.head(3)

,translation,laser_score,source_sentence_lid,target_sentence_lid
0,"{'eng_Latn': 'than for their Lord.""', 'npi_Dev...",1.249867,0.99996,0.78533
1,"{'eng_Latn': 'Indeed, for he who fears the pun...",1.249681,0.99999,0.99941
2,"{'eng_Latn': 'On that day man shall cry, ""Wher...",1.249663,0.99998,0.99986


#### Filtering such that only lid >= 1.0

As language id describe how close the sentence is to the source and target language, we will use only those having lid >= 1.

In [8]:
df_500000.describe()

,laser_score,source_sentence_lid,target_sentence_lid
count,500000.000000,500000.000000,500000.000000
mean,1.114876,0.998235,0.969973
std,0.016559,0.005982,0.089895
min,1.099634,0.950000,0.500050
25%,1.103760,0.999610,0.997550
50%,1.109666,0.999970,0.999930
75%,1.120159,1.000010,1.000010
max,1.249867,1.000010,1.000010


In [9]:
df_500000 = df_500000[(df_500000['source_sentence_lid'] >= 1.0) & (df_500000['target_sentence_lid'] >= 1.0)]

# Display the first 3 rows of the filtered DataFrame
print(df_500000.head(3))

                                          translation  laser_score  \
14  {'eng_Latn': 'The likes of which had not been ...     1.249197   
19  {'eng_Latn': 'Ilyas was also from the messenge...     1.248864   
21  {'eng_Latn': 'So did not benefit them whatever...     1.248716   

    source_sentence_lid  target_sentence_lid  
14              1.00001              1.00000  
19              1.00001              1.00000  
21              1.00001              1.00001  


In [10]:
df_500000.shape

(87992, 4)

- Here, we reduced to 87992 sentences. So, we dropped by quite a significant amount.

#### Function to calculate Cosine Similarity

- Now, upon closer inspection, we found that there are many misaligned translations. So, we will delete those as well.
- For that, we will be using Cosine similarity to compare the similarity of source and target sentences.
- We will be using the LaBSE model.
- For testing, we have first used 1000 rows and accepting any translations above 0.7 cosine similarity.

In [11]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the multilingual sentence transformer model (LaBSE or similar)
model = SentenceTransformer('sentence-transformers/LaBSE')

# Example sentences in English and Nepali
english_sentence = "The book is on the table."
nepali_sentence = "पुस्तक तेबल माथी छ ।"

# Convert sentences to embeddings
embedding_english = model.encode([english_sentence])
embedding_nepali = model.encode([nepali_sentence])

# Compute cosine similarity
similarity = cosine_similarity(embedding_english, embedding_nepali)

print(f"Cosine Similarity: {similarity[0][0]}")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/461 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/2.22k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/2.36M [00:00<?, ?B/s]

2_Dense/config.json:   0%|          | 0.00/114 [00:00<?, ?B/s]

Cosine Similarity: 0.8156033754348755


#### Slower Version Cosine Sim filter

In [ ]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time

# Load the multilingual sentence transformer model (LaBSE or similar)
model = SentenceTransformer('sentence-transformers/LaBSE')

# Assuming you have df_1000000 loaded already with columns 'source_sentence' and 'target_sentence'
# Filter the first 1000 rows
df_1000_temp = filtered_lid_df_500000.head(1000)

# List to hold the filtered sentences
filtered_sentences = []

# Start measuring time
start_time = time.time()

# Loop through each row and calculate cosine similarity
for index, row in filtered_lid_df_500000.iterrows():
    english_sentence = row['translation']['eng_Latn']
    nepali_sentence = row['translation']['npi_Deva']

    # Convert sentences to embeddings
    embedding_english = model.encode([english_sentence])
    embedding_nepali = model.encode([nepali_sentence])

    # Compute cosine similarity
    similarity = cosine_similarity(embedding_english, embedding_nepali)[0][0]

    # Filter if similarity > 0.5
    if similarity > 0.5:
        filtered_sentences.append(row)  # Add the row to the filtered list

# Convert the filtered list back to a DataFrame
filtered_df_init = pd.DataFrame(filtered_sentences)

# End time and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Output the time it took
print(f"Time taken to filter 1000 rows: {elapsed_time:.2f} seconds")

# Show the filtered DataFrame
print(filtered_df_init.head())


#### Faster Version

In [12]:
df_500000.drop(columns=['laser_score', 'source_sentence_lid', 'target_sentence_lid'], inplace=True)

In [13]:
df_500000.shape

(87992, 1)

##### Test Only

In [14]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import time
from concurrent.futures import ThreadPoolExecutor
import ast

# Load the multilingual sentence transformer model (LaBSE or similar)
model = SentenceTransformer('sentence-transformers/LaBSE')

# Assuming you have df_500000 loaded already with columns dict with keys 'eng_Latn' and 'npi_Deva'
df_1000_temp = df_500000.head(1000)

# Start measuring time
start_time = time.time()

# Batch encode the sentences
english_sentences = df_1000_temp['translation'].apply(lambda x: ast.literal_eval(x)['eng_Latn']).tolist()
nepali_sentences = df_1000_temp['translation'].apply(lambda x: ast.literal_eval(x)['npi_Deva']).tolist()

# Convert sentences to embeddings in batches
embedding_english = model.encode(english_sentences, convert_to_tensor=True)
embedding_nepali = model.encode(nepali_sentences, convert_to_tensor=True)

# Compute cosine similarity in batches
cosine_similarities = cosine_similarity(embedding_english.cpu().numpy(), embedding_nepali.cpu().numpy())

# Filter rows where similarity > 0.7
filtered_sentences = df_1000_temp[cosine_similarities.diagonal() > 0.7]

# End time and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Output the time it took
print(f"Time taken to filter 1000 rows: {elapsed_time:.2f} seconds")

# Show the filtered DataFrame
print(filtered_sentences.head())


Time taken to filter 1000 rows: 2.99 seconds
                                          translation
19  {'eng_Latn': 'Ilyas was also from the messenge...
31  {'eng_Latn': "And when the truth came to them,...
34  {'eng_Latn': 'Show me what they have created o...
45  {'eng_Latn': 'This is not hard for God.', 'npi...
71  {'eng_Latn': 'That is not a hard thing for God...


In [15]:
filtered_sentences.shape

(416, 1)

- So, accepting translations with quality of greater than 0.7 cosine similarity.
- Out of every 1000 rows, approx 416 rows are good.
- Likewise, it takes 193 seconds approx. to filter 1000 rows. so, it will be quite a bit.

##### Use this below

In [14]:
df_500000.reset_index(drop=True, inplace=True)

In [15]:
df_500000.shape

(87992, 1)

##### Use this instead

In [16]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import ast
import time

# Load the multilingual sentence transformer model
model = SentenceTransformer('sentence-transformers/LaBSE')

# Start measuring time
start_time = time.time()

# Define batch size
batch_size = 1000  # Adjust based on available memory and compute power

# Extract English and Nepali sentences
english_sentences = df_500000['translation'].apply(lambda x: ast.literal_eval(x)['eng_Latn']).tolist()
nepali_sentences = df_500000['translation'].apply(lambda x: ast.literal_eval(x)['npi_Deva']).tolist()

# Initialize lists for filtered rows
filtered_indices = []

# Process in batches
for i in range(0, len(english_sentences), batch_size):
    # Get batch
    english_batch = english_sentences[i:i + batch_size]
    nepali_batch = nepali_sentences[i:i + batch_size]

    # Encode sentences to embeddings
    embedding_english = model.encode(english_batch, convert_to_tensor=True)
    embedding_nepali = model.encode(nepali_batch, convert_to_tensor=True)

    # Normalize embeddings
    embedding_english = torch.nn.functional.normalize(embedding_english, p=2, dim=1)
    embedding_nepali = torch.nn.functional.normalize(embedding_nepali, p=2, dim=1)

    # Compute cosine similarity for the batch
    similarities = torch.sum(embedding_english * embedding_nepali, dim=1).cpu().numpy()  # Faster diagonal

    # Filter rows with similarity > 0.8
    batch_filtered_indices = np.where(similarities > 0.8)[0] + i
    filtered_indices.extend(batch_filtered_indices)

# Get the filtered DataFrame
filtered_sentences = df_500000.iloc[filtered_indices]

# End time and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Output the time it took
print(f"Time taken to filter {df_500000.shape[0]} rows: {elapsed_time:.2f} seconds")

# Show the filtered DataFrame
print(filtered_sentences.head())


Time taken to filter 87992 rows: 229.96 seconds
                                          translation
5   {'eng_Latn': 'Show me what they have created o...
15  {'eng_Latn': 'He was one of Our faithful serva...
21  {'eng_Latn': 'And that his effort will soon be...
36  {'eng_Latn': 'We rescued Moses and all those w...
85  {'eng_Latn': '24 Even us, whom also he hath ca...


In [17]:
filtered_sentences.shape

(40477, 1)

So, we got 40477 out of 500000 good translations.

In [19]:
filtered_sentences.shape[0]

40477

In [18]:
filtered_sentences.iloc[10]['translation']

'{\'eng_Latn\': "This is the Grace of Allah to us and to mankind, but most of the people don\'t thank.", \'npi_Deva\': \'यो अल्लाहको अनुकम्पा हो, हामीमाथि पनि र अरु मानिसहरू माथि पनि, तर धेरै मानिसहरूले आभार प्रकट गर्दैनन् ।\'}'

Random sample of 100 rows shows good translation quality.

- Upon close inspection, we saw that there are many rows where english numerals aren't present but, Nepali numerals are.
- So, the function below removes any occurance of nepali numerals if it isn't present in the english translation.

In [22]:
print(filtered_sentences.iloc[70]['translation'])

{'eng_Latn': 'At that time, it was often used to defend the city.', 'npi_Deva': 'त्यो समयमा, यो अक्सर शहर रक्षा गर्न प्रयोग गरिएको थियो.'}


In [23]:
import re

def remove_extra_numerals_and_brackets(english_sentence, nepali_sentence):
    """
    Removes numerals from the Nepali sentence if their corresponding English numerals are not present in the English sentence.
    Also removes all brackets from both sentences.

    Args:
        english_sentence: The English sentence.
        nepali_sentence: The Nepali sentence.

    Returns:
        A tuple containing the modified English sentence and the filtered Nepali sentence.
    """
    # Mapping of English numerals to Devanagari numerals
    numeral_mapping = {
        '0': '०', '1': '१', '2': '२', '3': '३', '4': '४',
        '5': '५', '6': '६', '7': '७', '8': '८', '9': '९'
    }

    # Remove brackets from both sentences
    english_sentence = re.sub(r'[()]', '', english_sentence)
    nepali_sentence = re.sub(r'[()]', '', nepali_sentence)

    # Extract English numerals and their corresponding Devanagari equivalents
    eng_numerals_and_brackets = set(re.findall(r'[\d)]', english_sentence))

    dev_numerals_and_brackets = set()
    for char in eng_numerals_and_brackets:
        if char.isdigit():
            dev_numerals_and_brackets.add(numeral_mapping.get(char, char))  # Map to Devanagari if possible
        else:
            dev_numerals_and_brackets.add(char)

    # Filter out Devanagari numerals and brackets that are not in the Nepali sentence
    filtered_nepali_sentence = ''.join([char for char in nepali_sentence if char in dev_numerals_and_brackets or not char.isdigit()])

    return english_sentence, filtered_nepali_sentence  # Return both modified sentences

# Example sentences
english_sentence = "1) He was one of Our faithful servants."
nepali_sentence = '८१) उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।'

# Apply the function
modified_english, filtered_nepali = remove_extra_numerals_and_brackets(english_sentence, nepali_sentence)

print("Modified English Sentence:", modified_english)  # Output: Modified English Sentence: 1 He was one of Our faithful servants.
print("Filtered Nepali Sentence:", filtered_nepali)    # Output: Filtered Nepali Sentence: १ उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।

Modified English Sentence: 1 He was one of Our faithful servants.
Filtered Nepali Sentence: १ उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।


In [27]:
import ast
filtered_sentences['modified_translation'] = filtered_sentences['translation'].apply(lambda x: remove_extra_numerals_and_brackets(ast.literal_eval(x)['eng_Latn'], ast.literal_eval(x)['npi_Deva']))

# Extract the modified English and Nepali sentences into separate columns
filtered_sentences[['target', 'source']] = pd.DataFrame(filtered_sentences['modified_translation'].tolist(), index=filtered_sentences.index)

# Drop the temporary 'modified_translation' column if needed
filtered_sentences = filtered_sentences.drop(columns=['modified_translation'])

# Display the updated DataFrame
filtered_sentences.head(1)

,translation,target,source
5,{'eng_Latn': 'Show me what they have created o...,Show me what they have created of the earth; o...,""" मलाई देखाऊ कि उनीहरूले धरतीको कुनचाहिँ भाग स..."


In [28]:
filtered_sentences.head()

,translation,target,source
5,{'eng_Latn': 'Show me what they have created o...,Show me what they have created of the earth; o...,""" मलाई देखाऊ कि उनीहरूले धरतीको कुनचाहिँ भाग स..."
15,{'eng_Latn': 'He was one of Our faithful serva...,He was one of Our faithful servants.,उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।
21,{'eng_Latn': 'And that his effort will soon be...,And that his effort will soon be seen.,र यो कि उसको प्रयास शीघ्र नै देखिनेछ ।
36,{'eng_Latn': 'We rescued Moses and all those w...,We rescued Moses and all those who were with him.,"र हामीले ""मूसा"" र उनको साथमा भएका सबैलाई बचाइ..."
85,"{'eng_Latn': '24 Even us, whom also he hath ca...","24 Even us, whom also he hath called, not only...","२४ अर्थात् हामीलाई, जसलाई उहाँले यहूदीहरूको मा..."


In [29]:
filtered_sentences.drop_duplicates(subset=['target', 'source'], inplace=True)

In [30]:
filtered_sentences.head()

,translation,target,source
5,{'eng_Latn': 'Show me what they have created o...,Show me what they have created of the earth; o...,""" मलाई देखाऊ कि उनीहरूले धरतीको कुनचाहिँ भाग स..."
15,{'eng_Latn': 'He was one of Our faithful serva...,He was one of Our faithful servants.,उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।
21,{'eng_Latn': 'And that his effort will soon be...,And that his effort will soon be seen.,र यो कि उसको प्रयास शीघ्र नै देखिनेछ ।
36,{'eng_Latn': 'We rescued Moses and all those w...,We rescued Moses and all those who were with him.,"र हामीले ""मूसा"" र उनको साथमा भएका सबैलाई बचाइ..."
85,"{'eng_Latn': '24 Even us, whom also he hath ca...","24 Even us, whom also he hath called, not only...","२४ अर्थात् हामीलाई, जसलाई उहाँले यहूदीहरूको मा..."


In [31]:
filtered_sentences.shape

(40471, 3)

In [32]:
filtered_sentences.drop(columns=['translation'], inplace=True)

In [33]:
filtered_sentences.reset_index(drop=True, inplace=True)

In [34]:
filtered_sentences.head()

,target,source
0,Show me what they have created of the earth; o...,""" मलाई देखाऊ कि उनीहरूले धरतीको कुनचाहिँ भाग स..."
1,He was one of Our faithful servants.,उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।
2,And that his effort will soon be seen.,र यो कि उसको प्रयास शीघ्र नै देखिनेछ ।
3,We rescued Moses and all those who were with him.,"र हामीले ""मूसा"" र उनको साथमा भएका सबैलाई बचाइ..."
4,"24 Even us, whom also he hath called, not only...","२४ अर्थात् हामीलाई, जसलाई उहाँले यहूदीहरूको मा..."


In [35]:
filtered_sentences.to_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/cleaned-nllb/cleaned-nllb-subset-1.csv')

## Subset 2 And Onwards

#### LOAD

In [1]:
import pandas as pd

In [66]:
df_500000 = pd.read_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/nllb-subsets/nllb-subset-38.csv')

In [67]:
df_500000.head(3)

,Unnamed: 0,translation,laser_score,source_sentence_lid,target_sentence_lid
0,0,"{'eng_Latn': '1915 Indian Motorcycle.', 'npi_D...",1.050787,0.99088,0.72504
1,1,{'eng_Latn': '20% of young people living in NS...,1.050787,1.00000,0.99451
2,2,"{'eng_Latn': 'A few hours later, Kesselbach is...",1.050787,0.99982,1.00001


In [68]:
df_500000.columns

Index(['Unnamed: 0', 'translation', 'laser_score', 'source_sentence_lid',
       'target_sentence_lid'],
      dtype='object')

In [69]:
df_500000.drop(columns=['Unnamed: 0'], inplace=True)

In [70]:
df_500000.head(3)

,translation,laser_score,source_sentence_lid,target_sentence_lid
0,"{'eng_Latn': '1915 Indian Motorcycle.', 'npi_D...",1.050787,0.99088,0.72504
1,{'eng_Latn': '20% of young people living in NS...,1.050787,1.00000,0.99451
2,"{'eng_Latn': 'A few hours later, Kesselbach is...",1.050787,0.99982,1.00001


In [71]:
df_500000.shape

(500000, 4)

#### Filtering such that only lid >= 1.0

As language id describe how close the sentence is to the source and target language, we will use only those having lid >= 1.

In [72]:
df_500000.describe()

,laser_score,source_sentence_lid,target_sentence_lid
count,500000.000000,500000.000000,500000.000000
mean,1.050611,0.997288,0.966846
std,0.000101,0.007530,0.093073
min,1.050437,0.950000,0.500010
25%,1.050524,0.999100,0.995210
50%,1.050611,0.999920,0.999840
75%,1.050699,1.000000,1.000000
max,1.050787,1.000010,1.000010


In [73]:
df_500000 = df_500000[(df_500000['source_sentence_lid'] >= 1.0) & (df_500000['target_sentence_lid'] >= 1.0)]

# Display the first 3 rows of the filtered DataFrame
print(df_500000.head(3))

                                          translation  laser_score  \
10  {'eng_Latn': 'And they had no idea that they w...     1.050787   
24  {'eng_Latn': 'District governments are also en...     1.050787   
35  {'eng_Latn': "For that matter, if he was up to...     1.050787   

    source_sentence_lid  target_sentence_lid  
10              1.00001              1.00001  
24              1.00000              1.00001  
35              1.00001              1.00001  


In [74]:
df_500000.shape

(64092, 4)

#### Calculate Cosine Similarity

In [75]:
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# Load the multilingual sentence transformer model (LaBSE or similar)
model = SentenceTransformer('sentence-transformers/LaBSE')

# Example sentences in English and Nepali
english_sentence = "The book is on the table."
nepali_sentence = "पुस्तक तेबल माथी छ ।"

# Convert sentences to embeddings
embedding_english = model.encode([english_sentence])
embedding_nepali = model.encode([nepali_sentence])

# Compute cosine similarity
similarity = cosine_similarity(embedding_english, embedding_nepali)

print(f"Cosine Similarity: {similarity[0][0]}")


Cosine Similarity: 0.8156033754348755


#### Faster Version

In [76]:
df_500000.drop(columns=['laser_score', 'source_sentence_lid', 'target_sentence_lid'], inplace=True)

In [77]:
df_500000.shape

(64092, 1)

##### Use this below

In [78]:
df_500000.reset_index(drop=True, inplace=True)

In [79]:
df_500000.shape

(64092, 1)

In [80]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import torch
import numpy as np
import ast
import time

# Load the multilingual sentence transformer model
model = SentenceTransformer('sentence-transformers/LaBSE')

# Start measuring time
start_time = time.time()

# Define batch size
batch_size = 1000  # Adjust based on available memory and compute power

# Extract English and Nepali sentences
english_sentences = df_500000['translation'].apply(lambda x: ast.literal_eval(x)['eng_Latn']).tolist()
nepali_sentences = df_500000['translation'].apply(lambda x: ast.literal_eval(x)['npi_Deva']).tolist()

# Initialize lists for filtered rows
filtered_indices = []

# Process in batches
for i in range(0, len(english_sentences), batch_size):
    # Get batch
    english_batch = english_sentences[i:i + batch_size]
    nepali_batch = nepali_sentences[i:i + batch_size]

    # Encode sentences to embeddings
    embedding_english = model.encode(english_batch, convert_to_tensor=True)
    embedding_nepali = model.encode(nepali_batch, convert_to_tensor=True)

    # Normalize embeddings
    embedding_english = torch.nn.functional.normalize(embedding_english, p=2, dim=1)
    embedding_nepali = torch.nn.functional.normalize(embedding_nepali, p=2, dim=1)

    # Compute cosine similarity for the batch
    similarities = torch.sum(embedding_english * embedding_nepali, dim=1).cpu().numpy()  # Faster diagonal

    # Filter rows with similarity > 0.8
    batch_filtered_indices = np.where(similarities > 0.8)[0] + i
    filtered_indices.extend(batch_filtered_indices)

# Get the filtered DataFrame
filtered_sentences = df_500000.iloc[filtered_indices]

# End time and calculate duration
end_time = time.time()
elapsed_time = end_time - start_time

# Output the time it took
print(f"Time taken to filter {df_500000.shape[0]} rows: {elapsed_time:.2f} seconds")

# Show the filtered DataFrame
print(filtered_sentences.head())


Time taken to filter 64092 rows: 151.93 seconds
                                          translation
25  {'eng_Latn': 'He has played so many games.', '...
26  {'eng_Latn': 'He is the 13th president of Indi...
42  {'eng_Latn': 'They do not mention anything abo...
46  {'eng_Latn': 'Those days were a lot of fun.', ...
55  {'eng_Latn': 'But it was happy, too.', 'npi_De...


In [81]:
filtered_sentences.shape

(8706, 1)

In [82]:
filtered_sentences.iloc[69]['translation']

"{'eng_Latn': 'You will also help to keep it healthy.', 'npi_Deva': 'यसले पनि तपाईको स्वास्थ ठीक राख्न केही मद्त गर्नेछ।'}"

Random sample of 100 rows shows good translation quality.

- Upon close inspection, we saw that there are many rows where english numerals aren't present but, Nepali numerals are.
- So, the function below removes any occurance of nepali numerals if it isn't present in the english translation.

In [83]:
print(filtered_sentences.iloc[0]['translation'])

{'eng_Latn': 'He has played so many games.', 'npi_Deva': 'यस्ता धेरै खेल उहाँले खेल्नुभएको छ ।'}


In [84]:
import re

def remove_extra_numerals_and_brackets(english_sentence, nepali_sentence):
    """
    Removes numerals from the Nepali sentence if their corresponding English numerals are not present in the English sentence.
    Also removes all brackets from both sentences.

    Args:
        english_sentence: The English sentence.
        nepali_sentence: The Nepali sentence.

    Returns:
        A tuple containing the modified English sentence and the filtered Nepali sentence.
    """
    # Mapping of English numerals to Devanagari numerals
    numeral_mapping = {
        '0': '०', '1': '१', '2': '२', '3': '३', '4': '४',
        '5': '५', '6': '६', '7': '७', '8': '८', '9': '९'
    }

    # Remove brackets from both sentences
    english_sentence = re.sub(r'[()]', '', english_sentence)
    nepali_sentence = re.sub(r'[()]', '', nepali_sentence)

    # Extract English numerals and their corresponding Devanagari equivalents
    eng_numerals_and_brackets = set(re.findall(r'[\d)]', english_sentence))

    dev_numerals_and_brackets = set()
    for char in eng_numerals_and_brackets:
        if char.isdigit():
            dev_numerals_and_brackets.add(numeral_mapping.get(char, char))  # Map to Devanagari if possible
        else:
            dev_numerals_and_brackets.add(char)

    # Filter out Devanagari numerals and brackets that are not in the Nepali sentence
    filtered_nepali_sentence = ''.join([char for char in nepali_sentence if char in dev_numerals_and_brackets or not char.isdigit()])

    return english_sentence, filtered_nepali_sentence  # Return both modified sentences

# Example sentences
english_sentence = "1) He was one of Our faithful servants."
nepali_sentence = '८१) उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।'

# Apply the function
modified_english, filtered_nepali = remove_extra_numerals_and_brackets(english_sentence, nepali_sentence)

print("Modified English Sentence:", modified_english)  # Output: Modified English Sentence: 1 He was one of Our faithful servants.
print("Filtered Nepali Sentence:", filtered_nepali)    # Output: Filtered Nepali Sentence: १ उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।

Modified English Sentence: 1 He was one of Our faithful servants.
Filtered Nepali Sentence: १ उनी हाम्रा ईमानवाला सेवकहरूमध्येका थिए ।


In [85]:
import ast
filtered_sentences['modified_translation'] = filtered_sentences['translation'].apply(lambda x: remove_extra_numerals_and_brackets(ast.literal_eval(x)['eng_Latn'], ast.literal_eval(x)['npi_Deva']))

# Extract the modified English and Nepali sentences into separate columns
filtered_sentences[['target', 'source']] = pd.DataFrame(filtered_sentences['modified_translation'].tolist(), index=filtered_sentences.index)

# Drop the temporary 'modified_translation' column if needed
filtered_sentences = filtered_sentences.drop(columns=['modified_translation'])

# Display the updated DataFrame
filtered_sentences.head(1)

<ipython-input-85-33dbe94132b2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sentences['modified_translation'] = filtered_sentences['translation'].apply(lambda x: remove_extra_numerals_and_brackets(ast.literal_eval(x)['eng_Latn'], ast.literal_eval(x)['npi_Deva']))
<ipython-input-85-33dbe94132b2>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_sentences[['target', 'source']] = pd.DataFrame(filtered_sentences['modified_translation'].tolist(), index=filtered_sentences.index)
<ipython-inp

,translation,target,source
25,"{'eng_Latn': 'He has played so many games.', '...",He has played so many games.,यस्ता धेरै खेल उहाँले खेल्नुभएको छ ।


In [86]:
filtered_sentences.head()

,translation,target,source
25,"{'eng_Latn': 'He has played so many games.', '...",He has played so many games.,यस्ता धेरै खेल उहाँले खेल्नुभएको छ ।
26,{'eng_Latn': 'He is the 13th president of Indi...,He is the 13th president of India.,उनी भारतको १३ औँ राष्ट्रपति भएका थिए ।
42,{'eng_Latn': 'They do not mention anything abo...,They do not mention anything about this in the...,उनीहरूको दस्ताबेजमा यस विषयमा कहीँ-कतै लेखिँदैन ।
46,"{'eng_Latn': 'Those days were a lot of fun.', ...",Those days were a lot of fun.,ती दिनहरु साच्चै रमाइला थिए ।
55,"{'eng_Latn': 'But it was happy, too.', 'npi_De...","But it was happy, too.","तर, खुशी पनि थियो ।"


In [87]:
filtered_sentences.shape

(8706, 3)

In [88]:
filtered_sentences.drop_duplicates(subset=['target', 'source'], inplace=True)

In [89]:
filtered_sentences.shape

(8706, 3)

In [90]:
filtered_sentences.head()

,translation,target,source
25,"{'eng_Latn': 'He has played so many games.', '...",He has played so many games.,यस्ता धेरै खेल उहाँले खेल्नुभएको छ ।
26,{'eng_Latn': 'He is the 13th president of Indi...,He is the 13th president of India.,उनी भारतको १३ औँ राष्ट्रपति भएका थिए ।
42,{'eng_Latn': 'They do not mention anything abo...,They do not mention anything about this in the...,उनीहरूको दस्ताबेजमा यस विषयमा कहीँ-कतै लेखिँदैन ।
46,"{'eng_Latn': 'Those days were a lot of fun.', ...",Those days were a lot of fun.,ती दिनहरु साच्चै रमाइला थिए ।
55,"{'eng_Latn': 'But it was happy, too.', 'npi_De...","But it was happy, too.","तर, खुशी पनि थियो ।"


In [91]:
filtered_sentences.shape

(8706, 3)

In [92]:
filtered_sentences.drop(columns=['translation'], inplace=True)

In [93]:
filtered_sentences.reset_index(drop=True, inplace=True)

In [94]:
filtered_sentences.head()

,target,source
0,He has played so many games.,यस्ता धेरै खेल उहाँले खेल्नुभएको छ ।
1,He is the 13th president of India.,उनी भारतको १३ औँ राष्ट्रपति भएका थिए ।
2,They do not mention anything about this in the...,उनीहरूको दस्ताबेजमा यस विषयमा कहीँ-कतै लेखिँदैन ।
3,Those days were a lot of fun.,ती दिनहरु साच्चै रमाइला थिए ।
4,"But it was happy, too.","तर, खुशी पनि थियो ।"


In [95]:
print(filtered_sentences.iloc[71]['target'])
print(filtered_sentences.iloc[71]['source'])


He said there is nothing like it in the world.
उनले भने, संसारमा केही पनि अजम्बरी छैन ।


In [96]:
filtered_sentences.to_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/cleaned-nllb/cleaned-nllb-subset-38.csv')

In [97]:
filtered_sentences.shape

(8706, 2)

## SLONE/NLLB-200-10M-Sample

### From HuggingFace

In [18]:
dataset = load_dataset("slone/nllb-200-10M-sample", "default")

In [19]:
dataset['train'][0]

{'laser_score': 1.1736073,
 'lang1': 'sot_Latn',
 'text1': 'Etsoe o tla pheteletsa mali a bahlanka ba hae.',
 'lang2': 'twi_Latn',
 'text2': 'ɛfiri sɛ, ɔbɛtɔ nʼasomfoɔ mogya so were.',
 'blaser_sim': 4.522867202758789}

In [ ]:
# Languages of interest
valid_languages = {'npi_Deva', 'eng_Latn'}

# Filter rows where both lang1 and lang2 are in valid_languages
filtered_data = [
    row for row in dataset['train']
    if row['lang1'] in valid_languages and row['lang2'] in valid_languages
]

print(len(filtered_data))

# Print the filtered dataset
# for row in filtered_data:
#     print(row)

In [23]:
import pandas as pd

# Assuming you have a DataFrame
df = pd.DataFrame(dataset['train'])




In [24]:
df.head(5)

,laser_score,lang1,text1,lang2,text2,blaser_sim
0,1.173607,sot_Latn,Etsoe o tla pheteletsa mali a bahlanka ba hae.,twi_Latn,"ɛfiri sɛ, ɔbɛtɔ nʼasomfoɔ mogya so were.",4.522867
1,1.113213,eng_Latn,Side note: you can do it yourself too.,lim_Latn,Meej aandere woorde: Ge kan zelf ôôk meej doen.,2.987574
2,1.156768,cjk_Latn,"Linalumbunuka ngwo, Zambi mahasa kumanununa yi...",fon_Latn,Ha nɛ waa hyɛ bɔnɛ Mawu pee enɛ ɔ ha.,2.606988
3,1.102681,lug_Latn,"Akagira ati: ""Obukama bwawe bwije; ebi okunda ...",wol_Latn,"Dafa leen ne: ""Na sa nguur ñëw, na sa coobare ...",4.242124
4,1.209065,bem_Latn,"""Talepuswa pe tempele, alebomba umulimo wa mus...",xho_Latn,"'Zange aphoswe kukuba setempileni, enikela ink...",4.605998


In [26]:
# Filter the DataFrame
valid_languages = ['npi_Deva', 'eng_Latn']
filtered_df = df[df['lang1'].isin(valid_languages) & df['lang2'].isin(valid_languages)]

In [27]:
filtered_df.head()

,laser_score,lang1,text1,lang2,text2,blaser_sim
1793,1.195897,eng_Latn,The Àd (people) rejected the messengers.,npi_Deva,अगमवक्तालाई अस्वीकार,3.076165
4856,1.198470,eng_Latn,deed) because he considers himself self-suffic...,npi_Deva,आफ्नो मान्छे आफ्नो हुन्छ भन्ने सोची काम गर्दा,2.690499
6376,1.229387,eng_Latn,Till this day they are terrified.,npi_Deva,उनीहरू त्यही दिनबाट डराउँछन् ।,3.924606
7680,1.208032,eng_Latn,"""Am I not better than this one who is despised...",npi_Deva,५२) बरु म त्यो व्यक्ति जसको कुनै इज्जत छैन र स...,3.301734
14073,1.230190,eng_Latn,"So, on this Day (of Judgement) the Believers w...",npi_Deva,३४) तसर्थ आज मोमिनहरूले ती काफिरहरूमाथि हाँसो ...,3.862711


In [28]:
filtered_df.to_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/nllb_eng-npi.csv')

### LOCAL

In [99]:
filtered_df_SLONE = pd.read_csv('/content/drive/MyDrive/Nepali-English-Translation/Dataset/nllb_eng-npi.csv')
len(filtered_df_SLONE)

3000

In [100]:
filtered_df_SLONE

,Unnamed: 0,laser_score,lang1,text1,lang2,text2,blaser_sim
0,1793,1.195897,eng_Latn,The Àd (people) rejected the messengers.,npi_Deva,अगमवक्तालाई अस्वीकार,3.076165
1,4856,1.198470,eng_Latn,deed) because he considers himself self-suffic...,npi_Deva,आफ्नो मान्छे आफ्नो हुन्छ भन्ने सोची काम गर्दा,2.690499
2,6376,1.229387,eng_Latn,Till this day they are terrified.,npi_Deva,उनीहरू त्यही दिनबाट डराउँछन् ।,3.924606
3,7680,1.208032,eng_Latn,"""Am I not better than this one who is despised...",npi_Deva,५२) बरु म त्यो व्यक्ति जसको कुनै इज्जत छैन र स...,3.301734
4,14073,1.230190,eng_Latn,"So, on this Day (of Judgement) the Believers w...",npi_Deva,३४) तसर्थ आज मोमिनहरूले ती काफिरहरूमाथि हाँसो ...,3.862711
...,...,...,...,...,...,...,...
2995,9977185,1.208144,eng_Latn,"""I have set before you good and evil.",npi_Deva,'मैले तिमीहरूलाई अकल्पनीय र कठिन काम नै सुम्पे...,3.263067
2996,9981403,1.209930,eng_Latn,"And they will know that God governs of Jacob, ...",npi_Deva,र तिनीहरूले परमेश्वरले याकूबलाई शासन गर्नेछ भन...,4.246202
2997,9982328,1.229124,eng_Latn,"Last year you visited a few Asian countries, [...",npi_Deva,गएको वर्ष तपाईले केही एसियाली देशहरूको भ्रमण ग...,3.698037
2998,9982549,1.228320,eng_Latn,"And Pharaoh said unto him, Get thee from me, t...",npi_Deva,७९) र फिरऔनले आदेश दियोः कि सबै अनुभवी जादूगरह...,3.136638


In [107]:
filtered_df_SLONE.drop(columns=['Unnamed: 0', 'laser_score', 'lang1', 'lang2', 'blaser_sim'], inplace=True)

In [109]:
filtered_df_SLONE.iloc[3]['text1']

'"Am I not better than this one who is despised and can barely be understood?"'

In [34]:
filtered_df.iloc[3]['text2']

'५२) बरु म त्यो व्यक्ति जसको कुनै इज्जत छैन र स्पष्ट बोल्न पनि सक्दैन उसभन्दा राम्रो छु ।'

In [118]:
filtered_df_SLONE.iloc[0]

,0
text1,The Àd (people) rejected the messengers.
text2,अगमवक्तालाई अस्वीकार


In [119]:
import ast
# Ensure 'text1' and 'text2' columns are present before applying the function.
# If they were dropped, you might need to reload the dataframe.

# Assuming the columns are still present:
filtered_df_SLONE['modified_translation'] = filtered_df_SLONE.apply(
    lambda x: remove_extra_numerals_and_brackets(x['text1'], x['text2']), axis=1
)  # axis=1 to apply function row-wise

# Extract the modified English and Nepali sentences into separate columns
filtered_df_SLONE[['target', 'source']] = pd.DataFrame(
    filtered_df_SLONE['modified_translation'].tolist(), index=filtered_df_SLONE.index
)  # Use filtered_df_SLONE.index for correct alignment

# Drop the temporary 'modified_translation' column if needed
filtered_sentences = filtered_df_SLONE.drop(columns=['modified_translation'])

# Display the updated DataFrame
filtered_sentences.head(1)

,text1,text2,target,source
0,The Àd (people) rejected the messengers.,अगमवक्तालाई अस्वीकार,The Àd people rejected the messengers.,अगमवक्तालाई अस्वीकार


In [120]:
len(filtered_sentences)

3000

In [121]:
filtered_sentences.drop(columns=['text1', 'text2'], inplace=True)

In [122]:
filtered_sentences.head()

,target,source
0,The Àd people rejected the messengers.,अगमवक्तालाई अस्वीकार
1,deed because he considers himself self-suffici...,आफ्नो मान्छे आफ्नो हुन्छ भन्ने सोची काम गर्दा
2,Till this day they are terrified.,उनीहरू त्यही दिनबाट डराउँछन् ।
3,"""Am I not better than this one who is despised...",बरु म त्यो व्यक्ति जसको कुनै इज्जत छैन र स्पष...
4,"So, on this Day of Judgement the Believers wil...",तसर्थ आज मोमिनहरूले ती काफिरहरूमाथि हाँसो गर्...
